In [86]:
# 导入相关包
import re as re
import numpy as np
import math as mt
import matplotlib.pyplot as plt

In [9]:
# 提取能量集中度txt文件中指定视场数据
# 参数：path -> 能量集中度txt文件保存位置
#       field_list -> 指定需要提取数据的视场，默认提取所有视场
# 返回：字典类型 -> 视场和其对应能量集中度函数曲线
def extract_data(path, field_list=-1):
    f = open(path, encoding='utf-8')
    txtData = f.readlines()
    
    params = {}
    count = 0
    index =0
    isFindField = False
    
    for i in range(len(txtData)):
        find_field = re.findall(r"Field", txtData[i])
        if find_field:
            fieldData = []
            index = i
            count += 1
            field = "Field" + str(count)
            isFindField = True

        if (i >= index+4) and isFindField:
            find_data = re.findall(r"\d+\.?\d*", txtData[i])
            if find_data:
                fieldData.append(find_data)
            else:
                params[field] = np.array(fieldData, dtype=np.float64)

        if i == len(txtData) - 1:
            params[field] = np.array(fieldData, dtype=np.float64)

    # 将指定视场能量集中度数据提取出来
    paramsSub = {}
    if field_list == -1:
        return params
    else:
        for i in field_list:
            paramsSub[('Field'+str(i))] = params[('Field'+str(i))]
        
        return paramsSub

In [82]:
# 将能量集中度数据转换为单个点的能量
# 参数： energy_list -> 不同半径list对应的能量
#        X_half_range -> X方向取点的半长度
#        Y_half_range -> Y方向取点的半长度
#        step ->X和Y方向取点步长，默认0.05
# 返回： pointEnergy -> 各个坐标点的能量值
def energy_in_point(energy_list, X_half_range, Y_half_range, step=0.05):
    X_range = np.arange(-X_half_range, X_half_range, step)
    Y_range = np.arange(-Y_half_range, Y_half_range, step)
    radiusStep = energy_list[0,1] - energy_list[0,0]
    count = np.zeros(energy_list.shape[1], dtype=np.int16)
    # 计算每个环内坐标点的个数
    for i in X_range:
        for j in Y_range:
            r = mt.sqrt(mt.pow(i,2)+mt.pow(j,2))
            index = int(r/radiusStep)
            if index < count.shape[0]:
                count[index] += 1

    #计算每个坐标点的能量
    pointEnergy = np.zeros((X_range.shape[0], Y_range.shape[0]),dtype = np.float64)
    for i in range(X_range.shape[0]):
        for j in range(Y_range.shape[0]):
            r = mt.sqrt(mt.pow(X_range[i],2)+mt.pow(Y_range[j],2))
            index = int(r/radiusStep) + 1
            if index < count.shape[0]:
                pointEnergy[i,j] = (energy_list[1,index] - energy_list[1,index-1])/count[index-1]
    
    return pointEnergy

In [106]:
def energy_in_pixel(energy_list, X_pixel, Y_pixel):
    pixelEnergy = np.zeros((X_pixel, Y_pixel), dtype=np.float64)
    X_step = energy_list.shape[1] / X_pixel
    Y_step = energy_list.shape[0] / Y_pixel
    for i in range(Y_pixel):
        for j in range(X_pixel):
            pixelEnergy[i, j] = np.sum(energy_list[int(i*X_step):int((i+1)*X_step), int(j*Y_step):int((j+1)*Y_step)])
    
    return pixelEnergy

In [108]:
energyData = extract_data('data/test.txt', [1])
pointEnergy = energy_in_point(energyData['Field1'].T, 22.5, 22.5)
pixelEnergy = energy_in_pixel(pointEnergy, 3, 3)
pixelEnergy

# a = np.array([[1,2,3],
#             [1,2,3],
#             [1,2,3]])
# np.sum(a[1:3, 1:3])

array([[0.00799504, 0.05509077, 0.00807616],
       [0.05509077, 0.62944283, 0.05613008],
       [0.00807616, 0.05613008, 0.0081591 ]])